In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy, es usada para concatenar al nombre del indice principal previa inserción
* ahora, es la fecha en un formato requerido para ES

In [3]:
now = datetime.now()
format_ES = "%Y.%m.%d"
fecha_hoy = str(now.strftime(format_ES))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))

### Definiendo indice principal con fecha de hoy

In [4]:
indice = parametros.usuarios_tablero11_index 
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ES

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [6]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["tablero11.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero11"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero11.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)



ultima fecha para control de ejecucion: 2021-07-01 00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [62]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion',
                           'nombreInstitucionEd']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

In [63]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.location']=datos_semilla['gestion.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)




Lectura fuente xxxxxx

Realizando inserción

In [49]:
use_these_keys = ['gestion.numeroTicket'
                  , 'gestion.usuarioTicket'
                  , 'gestion.estadoTicket'
                  , 'gestion.categoria'
                  , 'gestion.subcategoria'
                  , 'gestion.fechaApertura'
                  , 'gestion.fechaCierre'
                  , 'gestion.responsable'
                  , 'gestion.location'
                  , 'gestion.canal'
                  , 'gestion.id_Beneficiario'
                  , 'gestion.resolucion'
                  , 'gestion.Tipo3'
                  , 'gestion.Tipo4'
                  , 'gestion.nombreInstitucionEd'
                  , 'gestion.abierto'
                  , 'gestion.nombreSede'
                  , 'gestion.COD_ISO'
                  , 'gestion.site_id'
                  , 'gestion.dptoGestion'
                  , 'gestion.muniGestion'
                  , 'gestion.energiadesc'
                  , 'gestion.gravedad'
                  , 'gestion.anyo'
                  , 'gestion.mes'
                  , 'gestion.dia'
                  , 'gestion.hora'
                  , 'gestion.min'
                  , 'gestion.seg'
                  , 'gestion.cod_servicio'
                  , 'gestion.codDaneMuni'
                  , 'gestion.nombre_centro_pob'
                  , 'gestion.codCentroPoblado'
                  , 'gestion.codDaneInstitucionEdu'
                  , 'gestion.tipoSitio'
                  , 'gestion.detalleSitio'
                  , 'gestion.energia'
                  , 'gestion.region'
                  , 'gestion.matricula'
                  , 'gestion.DDA'
                  , 'gestion.grupoDesc'
                  , 'gestion.estadoInstalacion'
                  , '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['gestion.site_id']) + '-' + str(document['gestion.numeroTicket']) + '-' + str(document['gestion.fechaApertura'])}",
                "_source": filterKeys(document),
            }

In [53]:
try:
    concat = pd.merge(datos_SM_interactions,datos_semilla, on=['id_Beneficiario'],how='inner')
    concat = concat.rename(columns={'id_Beneficiario' : 'gestion.id_Beneficiario','resolution':'gestion.resolucion','variable3':'gestion.Tipo3','variable4':'gestion.Tipo4','nombreInstitucionEd':'gestion.nombreInstitucionEd', 'open':'gestion.abierto'
                                    ,'open_time':'gestion.fechaApertura','close_time':'gestion.fechaCierre','nombreSede':'gestion.nombreSede'
                                    ,'site_id':'gestion.site_id','nombre_departamento':'gestion.dptoGestion'
                                    ,'nombre_municipio':'gestion.muniGestion','nombre_centro_pob':'gestion.nombre_centro_pob'
                                    ,'COD_ISO':'gestion.COD_ISO','codDanesede':'gestion.codDanesede',
                                    'energiadesc':'gestion.energiadesc'
                                    ,'cod_servicio':'gestion.cod_servicio'
                  , 'codDaneMuni':'gestion.codDaneMuni', 'nombre_centro_pob':'gestion.nombre_centro_pob', 'codCentroPoblado':'gestion.codCentroPoblado'
                  , 'codDaneInstitucionEdu':'gestion.codDaneInstitucionEdu', 'tipoSitio':'gestion.tipoSitio', 'detalleSitio':'gestion.detalleSitio'
                  , 'energia':'gestion.energia', 'region':'gestion.region', 'matricula':'gestion.matricula', 'DDA':'gestion.DDA'
                  , 'grupoDesc':'gestion.grupoDesc','estadoInstalacion':'gestion.estadoInstalacion'                                                                   
                                   })
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.replace("T"," ")
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.slice(stop=19)
    concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.replace("T"," ")
    concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.slice(stop=19)
    concat["gestion.anyo"] = concat["gestion.fechaApertura"].str[0:4]
    concat["gestion.mes"] = concat["gestion.fechaApertura"].str[5:7]
    concat["gestion.dia"] = concat["gestion.fechaApertura"].str[8:10]
    concat["gestion.hora"] = concat["gestion.fechaApertura"].str[11:13]
    concat["gestion.min"] = concat["gestion.fechaApertura"].str[14:16]
    concat["gestion.seg"] = concat["gestion.fechaApertura"].str[17:19]
    concat['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator(concat))
    print("Fecha: ", now,"- Interacciones insertadas en indice principal:",salida[0])
except Exception as e:
    print(e, "\nNinguna Interacción insertada en indice principal:")

    

Ninguna Interacción insertada en indice principal:


Actualizando fecha de control de ejecución

In [55]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = '2021-04-29T00:00:00'

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia-tablero11',
        body = { 'jerarquia-tablero11': 'jerarquia-tablero11','tablero11.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)


actualizada fecha control de ejecucion: 2021-07-01T18:00:00
